In [9]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
import qiskit.quantum_info as qi
from numpy import pi
import os
import pandas as pd
from skimage.metrics import structural_similarity as compare_ssim

In [35]:
def normal1D(N):
        a0 = 1 / np.sqrt(N)
        a1 = np.sqrt(2/N)

        result = np.zeros((N, N))

        for u in range(0, N):
            for n in range(0, N):
                coef = a1
                if u == 0:
                    coef = a0

                result[u, n] = coef * np.cos((2*n + 1) * u * pi / (2 * N))

        return result

def initState(qc, qubits, state, size):
    
    if (not state.any()):
        state = block = np.array([[1] * size] * size)
    vec = state.flatten()
    norm = np.absolute(vec).sum()
    normState = []
    for i in range(0, vec.size):
        normState.append(float((1 if vec[i] >= 0 else -1) * np.sqrt(np.absolute(vec[i]))) / np.sqrt(norm))
    try:
        qc.initialize(normState, qubits)
    except:
        print("An exception occurred") 
    return norm
    
def createBlockFromMeasurements(measurements, totalMeasurements, size, norm):
    block = np.array([[0] * size] * size)
    for (key, value) in measurements.items():
        index = int(key, 2)
#         print(int(index/size))
#         print(int(index % size))
              
#         print((50 * (value / totalMeasurements)))
        block[int(index / size)][index % size] =  int(norm) * (value / totalMeasurements)
#         print(block)
    return block
#     print(block)
    
def qctIteration(qc, qubits, size):
    qct = qi.Operator(np.array(normal1D(2**int(size/2))))
    qc.unitary(qct, range(0,int(size/2)), label="qct")
    qc.unitary(qct, range(int(size/2), size), label="qct")
    
def createClassicBlockFromMeasurements(measurements, size, state, dctMatrix):
    block = np.array([[0] * size] * size)
    m = DCT.convertImage(state, dctMatrix)
    for (key, value) in measurements.items():
        index = int(key, 2)
#         print(block)
#         print(dctMatrix)
#         print("-----")
#         print(DCT.convertImage(block, dctMatrix))
        block[int(index / size)][index % size] =  m[int(index / size)][index % size]
    
    return block

In [31]:
state = np.array([[42, 7, 12, 8],
                         [10, 1, 15, 2],
                         [10, -24, -11, 10],
                         [10, -9, -10, -42]
                         ])

size = 6
N = 2**int(size/2)
totalMeasurements = 2
testOutput = {'00': 255, '10': 269, '01': 252, '11': 248}
dctLossy = normal1D(N)

path = os.path.abspath("deers.jpg")
image = ImageReader.getImage(path, N)
# ImageReader.displayImage(image)
chopped = ImageReader.chopUpImage(image, N)
# qr = QuantumRegister(size, 'input')
# cr = ClassicalRegister(size, 'result')
# qc = QuantumCircuit(qr, cr)

# initState(qc, qr, state)
# qc.barrier()
# qctIteration(qc, qr, size)
# qc.measure(qr, cr)
# qc.draw()

# # Use Aer's qasm_simulator
# backend_sim = Aer.get_backend('qasm_simulator')

# # Execute the circuit on the qasm simulator.
# # We've set the number of repeats of the circuit
# # to be 1024, which is the default.
# job_sim = backend_sim.run(transpile(qc, backend_sim), shots=totalMeasurements)

# # Grab the results from the job.
# result_sim = job_sim.result()
# counts = result_sim.get_counts(qc)
# # print(counts)
# createBlockFromMeasurements(counts, totalMeasurements, size)

dctMatrix = np.array(normal1D(2**int(size/2)))
total = 0
print(len(chopped['pieces']))
for i in range(0, len(chopped['pieces'])):
    if (i % 1000 == 0):
        print(i)
    state = chopped['pieces'][i]
    qr = QuantumRegister(size, 'input')
    cr = ClassicalRegister(size, 'result')
    qc = QuantumCircuit(qr, cr)

    norm = initState(qc, qr, state, size)
    qc.barrier()
    qctIteration(qc, qr, size)
    qc.measure(qr, cr)
    qc.draw()

    # Use Aer's qasm_simulator
    backend_sim = Aer.get_backend('qasm_simulator')

    # Execute the circuit on the qasm simulator.
    # We've set the number of repeats of the circuit
    # to be 1024, which is the default.
    job_sim = backend_sim.run(transpile(qc, backend_sim), shots=totalMeasurements)

    # Grab the results from the job.
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)
    # print(counts)
    
#     chopped['pieces'][i] = createBlockFromMeasurements(counts, totalMeasurements, N, norm)
    chopped['pieces'][i] = createClassicBlockFromMeasurements(counts, N, state, dctMatrix)
    chopped['pieces'][i] = DCT.remakeImageImage(
        chopped['pieces'][i], dctLossy)
# print(chopped)
reconstructed = ImageReader.reconstructImage(chopped, N)
print(image)
print(reconstructed)
ImageReader.displayImage( image, "Original")
ImageReader.displayImage( reconstructed, "Compressed")

cv2.waitKey(0)
cv2.destroyAllWindows()


(488, 800)
6100
0
1000


KeyboardInterrupt: 

In [36]:
class DCT:

    def normal1D(N):
        a0 = 1 / np.sqrt(N)
        a1 = np.sqrt(2/N)

        result = np.zeros((N, N))

        for u in range(0, N):
            for n in range(0, N):
                coef = a1
                if u == 0:
                    coef = a0

                result[u, n] = coef * np.cos((2*n + 1) * u * pi / (2 * N))

        return result

    def getDCTBitcount(dct):
        max = np.amax(dct)
        return int(math.log2(max)) + 1

    def int1D(N, precision):
        a0 = 1 / math.sqrt(N)
        a1 = math.sqrt(2/N)

        result = np.zeros((N, N), dtype=int)

        for u in range(0, N):
            for n in range(0, N):
                coef = a1
                if u == 0:
                    coef = a0

                result[u, n] = int(
                    (2**precision) * coef * np.cos((2*n + 1) * u * pi / (2 * N)))

        return result

    def convertImage(image, dct, precision=None):
        if (precision):
            return (np.matmul(np.matmul(dct, image).astype(int) / 2**(precision), np.transpose(dct)).astype(int) / 2**(precision))
        return np.matmul(np.matmul(dct, image), np.transpose(dct))

    def convertImage1D(image, dct, precision=None):
        return np.matmul(dct, image)

    def convertImage2D(image, dct, precision=None):
        return np.matmul(np.matmul(dct, image), np.transpose(dct))

    def remakeImageImage(image, dct, precision=None):
        if (precision):
            return (np.matmul(np.matmul(np.transpose(dct).astype(int) / 2**(precision), image), dct).astype(int) / 2**(precision))
        return np.matmul(np.matmul(np.transpose(dct), image), dct)

    def removeSmallBits(chunk):
        chunk[abs(chunk) < 100] = 0
        return chunk


class ImageReader:

    def getImage(path, N):
        # Read Images
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        maxSize = tuple([(int(x) - (int(x) % N)) for x in gray.shape])
        maxSize = tuple(reversed(maxSize))
        return cv2.resize(gray, maxSize)

    def displayImage(image, name):
        cv2.imshow(name, image)

    def chopUpImage(image, N):
        result = []
        size = image.shape
        print(size)

        for i in range(0, size[0], N):
            for j in range(0, size[1], N):
                result.append((image[i:i+N, j:j+N]).astype(int) - 128)

        return {
            "pieces": result,
            "size": size
        }

    def reconstructImage(pieces, N):
        length = pieces['size'][0]
        width = pieces['size'][1]
        vfunc = np.vectorize(setLimit)
        result = np.ndarray(shape=(length, width), dtype=np.uint8)
        index = 0
        for i in range(0, length, N):
            for j in range(0, width, N):
                # print((pieces['pieces'][index] + 128))
                result[i:i+N, j:j+N] = vfunc(pieces['pieces'][index] + 128)
                # print(result[i:i+N, j:j+N], i, j)
                index += 1
        
        return result
    TestImage = [os.path.abspath("deers.jpg")]
    ImageSetPaths = [
        "Image Test Sets\\artificial.pgm",
        "Image Test Sets\\big_building.pgm",
        "Image Test Sets\\big_tree.pgm",
        "Image Test Sets\\bridge.pgm",
        "Image Test Sets\cathedral.pgm",
        "Image Test Sets\deer.pgm",
        "Image Test Sets\\fireworks.pgm",
        "Image Test Sets\\flower_foveon.pgm",
        "Image Test Sets\hdr.pgm",
        "Image Test Sets\leaves_iso_200.pgm",
        "Image Test Sets\leaves_iso_1600.pgm",
        "Image Test Sets\\nightshot_iso_100.pgm",
        "Image Test Sets\\nightshot_iso_1600.pgm",
        "Image Test Sets\spider_web.pgm",
        "Image Test Sets\zone_plate.pgm",
    ]
        
def setLimit(a):
    a = a if a < 255 else 255
    return a if a > -255 else -255

In [11]:
from qiskit.test.mock import FakeCasablanca


In [33]:
def testImageSets(N):
    psnrs = []
    ssims = []
    dctLossy = DCT.normal1D(N)
    for path in ImageReader.ImageSetPaths:
        image = ImageReader.getImage(path, N)
        chopped = ImageReader.chopUpImage(image, N)
        print(len(chopped['pieces']))
        print("-----------")
        for i in range(0, len(chopped['pieces'])):
            if (i % 1000 == 0):
                print(i)
            state = chopped['pieces'][i]
            qr = QuantumRegister(size, 'input')
            cr = ClassicalRegister(size, 'result')
            qc = QuantumCircuit(qr, cr)

            norm = initState(qc, qr, state, size)
            qc.barrier()
            qctIteration(qc, qr, size)
            qc.measure(qr, cr)
            qc.draw()

            # Use Aer's qasm_simulator
            backend_sim = Aer.get_backend('qasm_simulator')

            # Execute the circuit on the qasm simulator.
            # We've set the number of repeats of the circuit
            # to be 1024, which is the default.
            job_sim = backend_sim.run(transpile(qc, backend_sim), shots=totalMeasurements)

            # Grab the results from the job.
            result_sim = job_sim.result()
            counts = result_sim.get_counts(qc)
            # print(counts)

        #     chopped['pieces'][i] = createBlockFromMeasurements(counts, totalMeasurements, N, norm)
            chopped['pieces'][i] = createClassicBlockFromMeasurements(counts, N, state, dctMatrix)
            chopped['pieces'][i] = DCT.remakeImageImage(
                chopped['pieces'][i], dctLossy)

        reconstructed = ImageReader.reconstructImage(chopped, N)
        psnr = cv2.PSNR(image, reconstructed)
        print("Image: {}".format(path))
        print("Chunk size: {}".format(N))
        print("PSNR: {}".format(psnr))
        psnrs.append(psnr)
        (score, diff) = compare_ssim(image, reconstructed, full=True)
        print("SSIM: {}\n".format(score))
        ssims.append(score)
    print("PSNR Mean:", np.array(psnrs).mean(),
          ". STD:", np.array(psnrs).std())
    print("SSIM Mean:", np.array(ssims).mean(),
          ". STD:", np.array(ssims).std(), "\n")


In [34]:
testImageSets(N)

(512, 768)
6144
-----------
0
1000
2000
3000
4000
5000
6000
Image: Image Test Sets\artificial.pgm
Chunk size: 8
PSNR: 21.71597298717859
SSIM: 0.6443070893206089

(1352, 1800)
38025
-----------
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
Image: Image Test Sets\big_building.pgm
Chunk size: 8
PSNR: 21.939488591458232
SSIM: 0.47268501107977334

(1136, 1520)
26980
-----------
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
Image: Image Test Sets\big_tree.pgm
Chunk size: 8
PSNR: 21.197484998447145
SSIM: 0.5725551658498759

(1008, 680)
10710
-----------
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
Image: Image Test Sets\bridge.pgm
Chunk size: 8
PSNR: 20.148114410187414
SSIM: 0.5340452888995304

(752, 496)
5828
-

In [ ]:
size = 6
N = 2**int(size/2)
testImageSets(N)

(2048, 3072)
98304
-----------
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
An exception occurred
An exception occurred
An exception occurred
An exception occurred
33000
An exception occurred
An exception occurred
An exception occurred
An exception occurred
34000
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
35000
An exception occurred
An exception occurred
An exception occurred
An exception occurred
36000
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
An exception occurred
37000
An exception occurred
An exception occurred
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
